# Build *Bacillus subtilis*

We will build an ME model from the NC_000913.2 Genbank file, the iJO1366
M model, and the complex reconstruction from iJL1678-ME

In [1]:
from __future__ import print_function, division, absolute_import

import sys
sys.path.append('/home/jt/UCSD/bacillusme-master/')

# python imports
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *

print(cobrame.__file__)
print(ecolime.__file__)

def print_rxn_info(me):
    last = 0
    reactionNumber = len(me.reactions);
    for i in range(last,reactionNumber):
        rxn = me.reactions[i]
        print(i)
        print(rxn)
        try:
            print(rxn.reaction)
        except:
            continue

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/UCSD/cobrame-master/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/ecolime/__init__.py


In [2]:
# Define Models
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
ijo = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

In [3]:
## M-model Fixes
ijo.reactions.BIOMASS_BS_10.lower_bound = 0
ijo.reactions.BIOMASS_BS_10.upper_bound = 0


## Part 1: Create minimum solveable ME-model
This will include the bare minimum representations of 
- Transcription Reactions
- Translation Reactions
- Complex Formation Reactions
- Metabolic Reactions

that still produce a working ME-model

### 1) Create Model Object and populate its global info
This includes important parameters that are used to calculate coupling 
constraints as well as organism-specific information such as peptide 
processing types

In [4]:
me = cobrame.MEModel('iYO844-ME')

# ME-models require special OptLang interface if cobrapy version >= 0.6.0
# If cannot import SymbolicParameter, assume using cobrapy
# versions <= 0.5.11
try:
    from optlang.interface import SymbolicParameter
except:
    pass
else:
    me.solver = me_model_interface

# "Translational capacity" of organism
me.global_info['kt'] = 4.5  # (in h-1)scott 2010, RNA-to-protein curve fit
me.global_info['r0'] = 0.087  # scott 2010, RNA-to-protein curve fit
me.global_info['k_deg'] = 1.0/5. * 60.0  # 1/5 1/min 60 min/h # h-1

# Molecular mass of RNA component of ribosome
me.global_info['m_rr'] = 1453. # in kDa

# Average molecular mass of an amino acid
me.global_info['m_aa'] = 109. / 1000.  # in kDa

# Proportion of RNA that is rRNA
me.global_info['f_rRNA'] = .86
me.global_info['m_nt'] = 324. / 1000.  # in kDa
me.global_info['f_mRNA'] = .02

# tRNA associated global information
me.global_info['m_tRNA'] = 25000. / 1000.  # in kDA
me.global_info['f_tRNA'] = .12

# Folding Properties
me.global_info['temperature'] = 37
me.global_info['propensity_scaling'] = .45

# DNA Replication Parameters
me.global_info['GC_fraction'] = 0.507896997096

# Define the types of biomass that will be synthesized in the model
me.add_biomass_constraints_to_model(["protein_biomass", "mRNA_biomass", "tRNA_biomass", "rRNA_biomass",
                                     "ncRNA_biomass", "DNA_biomass", "lipid_biomass", "constituent_biomass",
                                     "prosthetic_group_biomass", "peptidoglycan_biomass"])

# Define ME-model compartments
me.compartments = {"p": "Periplasm", "e": "Extra-organism", "c": "Cytosol", 
                   "im": 'Inner Membrane', 'om': "Outer Membrane", "mc": "ME-model Constraint",
                   "m": "Membrane"}

### 2) Load metabolites and build Metabolic reactions
The below reads in:
- Required 
 * **reaction_matrix.txt** (reaction matrix w/ reactions unlumped, metabolites renamed etc.)
 * **metabolites.txt** (metabolite properties)
 * **reactions.txt** (info on reversiblity, whether enzyme catalyzed etc.)
 * **m_to_me_mets.csv** (mapping of enzymes/complexes used in M-model to their ME-model compatible ID)
 * **protein_complexes.txt** (protein subunit stoichiometry of all complexes, used to identify metabolites as such)
 
It creates a new e coli M-model from this info then incorporates it into the ME-model using *add_m_model_content*. metabolites are added directly reactions are added as StoichiometricData

Metabolite types have different properties in an ME-model so enzyme complexes need added to the model as Complexes not Metabolites. Components in the E. coli M-model that are actually Complexes are compiled in *complex_list*

In [5]:
# m_model = flat_files.get_m_model()
m_model = flat_files.process_m_model(ijo, 'metabolites.txt',
                                     'm_to_me_mets.csv', 'reactions.txt',
                                     'reaction_matrix.txt',
                                     'protein_complexes.txt',
                                     defer_to_rxn_matrix=set())
m_model.reactions.EX_glc__D_e.id = 'EX_glc__D_e'
m_model.repair()
# some of the "metabolites" in iJO1366 "M" model are actually complexes. We pass those in
# so they get created as complexes, not metabolites.
complexes = flat_files.get_complex_subunit_stoichiometry(
    'protein_complexes.txt').keys()
complex_set = set(
    [i.id for i in m_model.metabolites if i.id.split('_mod_')[0] in complexes])
building.add_m_model_content(me, m_model, complex_metabolite_ids=complex_set)

/home/jt/UCSD/bacillusme-master/ecolime/flat_files.py:60 UserWarning: Complex (BSU34550-MONOMER) in complex_stoichiometry_file twice


BSU15920-MONOMER_mod_pan4p
BSU15920-MONOMER
BSU28500-MONOMER_mod_Oxidized
BSU28500-MONOMER


/home/jt/UCSD/bacillusme-master/ecolime/corrections.py:28 UserWarning: Creating new metabolite (4fe4s_c)
/home/jt/UCSD/bacillusme-master/ecolime/corrections.py:28 UserWarning: Creating new metabolite (2fe2s_c)


In [6]:
# This adds exchange reactions for metabolites not contained in iJO1366
# Some of these cannot be produced by the model so they are added here
exchange_list = ['4fe4s_c','2fe2s_c','udcpp_c','mn2_c','pydx5p_c','3fe4s','cl_c','sheme_c','cu_c',
                 'bmocogdp_c','dpm_c','thmpp_c','zn2_c','cbl1_c','cobalt2_c']

for met_id in exchange_list:
    r = cobrame.MEReaction("EX_" + met_id)
    me.add_reaction(r)
    r.reaction = met_id + " <=> "

unknown metabolite '3fe4s' created
unknown metabolite 'cl_c' created
unknown metabolite 'cu_c' created
unknown metabolite 'bmocogdp_c' created
unknown metabolite 'dpm_c' created
unknown metabolite 'cbl1_c' created


### 3) Add Transcription and Translation 
The below reads in:
- Required
 * **NC_000913.2.gb** (Genbank sequence annotation)
 * **ecolime/translation.py** (codon to tRNA mapping)
- Optional
 * **TUs_from_ecocyc.txt** (TU definitions, start/stop positions, etc.)
 * **ecolime/translation.py** (dictionary of gene to frameshift mutation)
 
To construct the bare minimimum components of a transcription and translation reactions. For example, transcription reactions at this point include nucleotides and the synthesized RNAs.

In [7]:
element_types = {'CDS', 'rRNA','tRNA', 'ncRNA','misc_RNA'}

In [8]:
gb_filename = join(flat_files.ecoli_files_dir,'NC_000964.gb')                                                                                    
tu_df = flat_files.get_tu_dataframe('TUs_from_bsubcyc.txt')

building.build_reactions_from_genbank(me, gb_filename, tu_df, verbose=False,
                                      frameshift_dict=translation.frameshift_dict,
                                      trna_to_codon=translation.trna_to_codon,
                                      element_types = element_types)



Processing BSU_misc_RNA_1 as ncRNA
Processing BSU_misc_RNA_2 as ncRNA
Processing BSU_misc_RNA_3 as ncRNA
Processing BSU_misc_RNA_4 as ncRNA
Processing BSU_misc_RNA_5 as ncRNA
Processing BSU_misc_RNA_6 as ncRNA
Processing BSU_misc_RNA_64 as ncRNA
Processing BSU_misc_RNA_7 as ncRNA
Processing BSU_misc_RNA_65 as ncRNA
Processing BSU_misc_RNA_66 as ncRNA
Processing BSU_misc_RNA_8 as ncRNA
Processing BSU_misc_RNA_9 as ncRNA
Processing BSU_misc_RNA_10 as ncRNA
Processing BSU_misc_RNA_11 as ncRNA
Processing BSU_misc_RNA_12 as ncRNA
Processing BSU_misc_RNA_13 as ncRNA
Processing BSU_misc_RNA_67 as ncRNA
Processing BSU_misc_RNA_14 as ncRNA
Processing BSU_misc_RNA_15 as ncRNA
Processing BSU_misc_RNA_16 as ncRNA
Processing BSU_misc_RNA_17 as ncRNA
Processing BSU_misc_RNA_18 as ncRNA
Processing BSU_misc_RNA_19 as ncRNA
Processing BSU_misc_RNA_68 as ncRNA
Processing BSU_misc_RNA_87 as ncRNA
Processing BSU_misc_RNA_20 as ncRNA
Processing BSU_misc_RNA_21 as ncRNA
Processing BSU_misc_RNA_22 as ncRNA
P

/home/jt/UCSD/cobrame-master/cobrame/core/reaction.py:852 UserWarning: RNA Polymerase () not found
/home/jt/UCSD/cobrame-master/cobrame/core/reaction.py:1093 UserWarning: ribosome not found
/home/jt/UCSD/cobrame-master/cobrame/core/reaction.py:1136 UserWarning: RNA_degradosome not found


In [9]:
me.reactions.get_by_id('translation_BSU22660')

<TranslationReaction translation_BSU22660 at 0x7f2ba37ef9e8>

### 4) Add in complex Formation without modifications (for now)

The below reads in:
- Required
 * **protein_complexes.txt** (Metabolic complexes' protein subunit stoichiometries)
 * **protein_modification.txt** (Type and number of modifications for each protein)

In [10]:
# complex_stoichiometry_dict is a dict of {'complex_id': [{'bnum' : count}]}
rna_components = {'BSU_misc_RNA_2','BSU_misc_RNA_35'} # component id should have 'RNA_ instead' of 'protein_'
# get Metabolic Complex composition from ECOLIme
complex_stoichiometry_dict = flat_files.get_complex_subunit_stoichiometry('protein_complexes.txt', rna_components)
# add complexes to model
complex_modification_dict = flat_files.get_complex_modifications('protein_modification.txt', 'protein_complexes.txt')
building.add_model_complexes(me, complex_stoichiometry_dict, complex_modification_dict, verbose=False)

# remove modifications. they will be added back in later
for data in me.complex_data:
    data.subreactions = {}
# add formation reactions for each of the ComplexDatas
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

Created <TranslatedGene protein_BSU29679 at 0x7f2ba6fff4a8> in <ComplexFormation formation_BSU29679-MONOMER at 0x7f2ba6fff550>
Created <TranslatedGene protein_G8J2-7 at 0x7f2ba5b916a0> in <ComplexFormation formation_MONOMER8J2-7 at 0x7f2ba5b91be0>
Created <TranslatedGene protein_G8J2-20 at 0x7f2ba5b50438> in <ComplexFormation formation_MONOMER8J2-49 at 0x7f2ba5b503c8>
Created <TranslatedGene protein_G8J2-176 at 0x7f2ba5bcde48> in <ComplexFormation formation_MONOMER8J2-60 at 0x7f2ba5bcdda0>
Created <TranslatedGene protein_G8J2-19 at 0x7f2ba38d7588> in <ComplexFormation formation_MONOMER8J2-48 at 0x7f2ba38d73c8>
Created <TranslatedGene protein_G8J2-18 at 0x7f2ba38d75f8> in <ComplexFormation formation_MONOMER8J2-47 at 0x7f2ba38d7898>
Created <TranslatedGene protein_G8J2-16 at 0x7f2ba38d7a20> in <ComplexFormation formation_MONOMER8J2-46 at 0x7f2ba38d74e0>
Created <TranslatedGene protein_G8J2-13 at 0x7f2ba38d7940> in <ComplexFormation formation_MONOMER8J2-45 at 0x7f2ba38d79e8>
Created <Tran

### 5) Add dummy reaction to model and unmodeled_protein_fraction

Includes the transcription, translation, complex_formation, and metabolic reaction. Sequence based on prevelance of each codon found in *E. coli*.
 - Required
     * [**codon_usage.csv**](http://openwetware.org/wiki/Escherichia_coli/Codon_usage) (codon prevelance)

In [11]:
seq = "ATG"
codons = pandas.read_csv(join(flat_files.ecoli_files_dir, "codon_usage.csv"), index_col=0)
for codon, row in codons.iterrows():
    if row.amino_acid == "Stop":
        continue
    seq += codon * int(row.per_1000 // 3)  # want roughly 300 aa
# get the most used stop codon
seq += codons[codons.amino_acid == "Stop"].sort_values("per_1000").index[-1]
building.add_dummy_reactions(me, seq, update=True)

# dummy proteins can be used to catalyze orphan reactions. This
# ensures that orphan reactions will not becore favored to fulfil 
# unmodeled protein fraction requirement 
rxn = cobrame.SummaryVariable('dummy_protein_to_mass')
me.add_reactions([rxn])
mass = me.metabolites.protein_dummy.formula_weight / 1000.  # in kDa
rxn.add_metabolites({'protein_biomass': -mass, 'protein_dummy': -1,
                     cobrame.Constraint('unmodeled_protein_biomass'): mass})

### 6) Assocated Complexes and build Metabolic Reactions 
- Required
    * **enzyme_reaction_association.txt**
    * **reactions.txt** (gives reaction name, reversiblity, source and whether reaction is spontaneous)


In [12]:
# associate reaction id with the old ME complex id (including modifications)
rxn_to_cplx_dict = flat_files.get_reaction_to_complex(m_model)
rxn_info = flat_files.get_reaction_info_frame('reactions.txt')

# Required to add dummy reaction as spontaneous reaction
rxn_info = rxn_info.append(pandas.Series({'description': 'dummy reaction', 'is_reversible': 0,
                                          'is_spontaneous':1}, name='dummy_reaction'))

building.add_reactions_from_stoichiometric_data(me, rxn_to_cplx_dict, rxn_info, update=True)

/home/jt/UCSD/cobrame-master/cobrame/util/building.py:717 UserWarning: Reaction (BIOMASS_BS_10) cannot carry flux


### 7) Incorporate remaining biomass constituents
There are leftover components from the *i*JO1366 biomass equation that either: 
1. have no mechanistic function in the model (glycogen)
2. are cofactors that are regenerated (nad)

Applies demands and coefficients from the *i*JO1366 biomass objective function

In [13]:
me.ngam = 9.
me.gam = 105.053483
me.unmodeled_protein_fraction = .36

biomass_constituents = {
    "10fthf_c": -0.000367,
    "peptido_BS_c": -.101817,
    "psetha_BS_c": -0.00056,
    "mql7_c":-0.000266,
}


rxn = cobrame.SummaryVariable('biomass_constituent_demand')
me.add_reactions([rxn])
rxn.add_metabolites(biomass_constituents)
constituent_mass = sum(me.metabolites.get_by_id(c).formula_weight / 1000. * -v
                     for c, v in biomass_constituents.items())
rxn.lower_bound = mu
rxn.upper_bound = mu
rxn.add_metabolites({me.metabolites.constituent_biomass: constituent_mass})

/home/jt/UCSD/cobrame-master/cobrame/core/model.py:103 UserWarning: Adding ATPM reaction to model
/home/jt/UCSD/cobrame-master/cobrame/core/model.py:86 UserWarning: Adding GAM reaction to model


 #### Lipid components
 Metabolites and coefficients from *i*JO1366 biomass objective function

In [14]:
# Identify lipid metabolites in biomass equation

lipid_demand = {}
for key, value in ijo.reactions.BIOMASS_BS_10.metabolites.items():
    if '_BS_c' in key.id and not key.id in biomass_constituents:
        lipid_demand[key.id] = abs(value)
                
for met, requirement in lipid_demand.items():
    component_mass = me.metabolites.get_by_id(met).formula_weight / 1000.
    rxn = cobrame.SummaryVariable('Demand_' + met)
    me.add_reactions([rxn])
    rxn.add_metabolites({met: -1 * requirement,
                         'lipid_biomass': component_mass * requirement})
    rxn.lower_bound = mu
    rxn.upper_bound = 1000.

# Kdo2lipid4
#requirement = 0.01945  # in mmol/gDW
#met = me.metabolites.get_by_id('kdo2lipid4_e')
#component_mass = met.formula_weight / 1000.
#rxn = cobrame.SummaryVariable('Demand_' + met.id)
#me.add_reactions([rxn])

#rxn.add_metabolites({met.id : -1.*requirement,
#                     'lipid_biomass': component_mass * requirement})
#rxn.lower_bound = mu
#rxn.upper_bound = mu

#### DNA Demand Requirements
Added based on growth rate dependent DNA levels as in [O'brien EJ et al 2013](https://www.ncbi.nlm.nih.gov/pubmed/24084808)

In [15]:
dna_demand_stoich, dna_demand_bound = ecolime.dna_replication.return_gr_dependent_dna_demand(
    me, me.global_info['GC_fraction'])

dna_replication = cobrame.SummaryVariable("DNA_replication")
me.add_reaction(dna_replication)
dna_replication.add_metabolites(dna_demand_stoich)
dna_biomass = cobrame.Constraint("DNA_biomass")

dna_mw = 0
dna_mw_no_ppi = ecolime.dna_replication.get_dna_mw_no_ppi_dict(me)
for met, value in me.reactions.DNA_replication.metabolites.items():
    if met.id != 'ppi_c':
        dna_mw -= value * dna_mw_no_ppi[met.id.replace('_c','')] / 1000.
        
dna_replication.add_metabolites({dna_biomass: dna_mw})
dna_replication.lower_bound = dna_demand_bound 
dna_replication.upper_bound = dna_demand_bound 


**Note**: From this point forward, executing every codeblock should result in a solveable ME-model

------

## Part 2: Add metastructures to solving ME-model
This includes:
1. ribosome
2. RNA polymerase
3. charged_tRNAs

Sometimes multiple entities can perform the same role. To prevent a combinatorial explosion of possibilities, we can create  "generic" version, where any of those entities can fill in.

In [16]:
for generic, components in generics.generic_dict.items():
    cobrame.GenericData(generic, me, components).create_reactions()

### 1) Add ribosome
This uses the ribosome composition definition in **ecolime/ribosome.py**

In [17]:
ecolime.ribosome.add_ribosome(me, verbose=False)

/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:256 UserWarning: met (mg2_c) does not have formula


### 2) Add charged tRNA reactions

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

Uses **trna_charging.py**

In [18]:
aa_synthetase_dict = ecolime.trna_charging.amino_acid_trna_synthetase
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

Generic charged tRNAs are added to translation reactions via SubreactionData below.

All new data added in this block contained in **ecolime/translation.py**

In [19]:
ecolime.translation.add_charged_trna_subreactions(me)
for data in me.translation_data:
    data.add_initiation_subreactions(start_codons=translation.translation_start_codons,
                                    start_subreactions=set(translation.initiation_subreactions.keys()))

    data.add_elongation_subreactions(elongation_subreactions=set(translation.elongation_subreactions.keys()))

    data.add_termination_subreactions(translation_terminator_dict=translation.translation_stop_dict)

/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:870 UserWarning: initiation subreaction Translation_initiation_factor_InfA not in model
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:870 UserWarning: initiation subreaction Translation_initiation_factor_InfC not in model
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:870 UserWarning: initiation subreaction fmet_addition_at_START not in model
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:870 UserWarning: initiation subreaction Translation_gtp_initiation_factor_InfB not in model
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:836 UserWarning: elongation subreaction Tuf_gtp_regeneration not in model
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:836 UserWarning: elongation subreaction FusA_mono_elongation not in model
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:863 UserWarning: RNA_BSU05343 starts with 'UGC' which is not a start codon
/home/jt/UCSD/cobrame-master/cobra

Adding selenocystein for BSU20040


/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:900 UserWarning: No termination enzyme for RNA_BSU23330
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:863 UserWarning: RNA_BSU23836 starts with 'UGC' which is not a start codon
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:900 UserWarning: No termination enzyme for RNA_BSU23836
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:863 UserWarning: RNA_BSU26039 starts with 'AGC' which is not a start codon
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:900 UserWarning: No termination enzyme for RNA_BSU26040
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:863 UserWarning: RNA_BSU26748 starts with 'AUA' which is not a start codon
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:900 UserWarning: No termination enzyme for RNA_BSU26749
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:900 UserWarning: No termination enzyme for RNA_BSU26750
/home/jt/UCSD/cobrame-master/cobrame/co

### 3) Add Transcription Metacomplexes
#### RNA Polymerase

Data for RNA_polymerase composition fround in **ecolime/transcription**

Uses *tu_df* from **TUs_from_ecocyc.txt**, above

In [20]:
for met in transcription.rna_polymerases:
    rnap_obj = cobrame.RNAP(met)
    me.add_metabolites(rnap_obj)
transcription.add_rna_polymerase_complexes(me, verbose=False)

# associate the correct RNA_polymerase and factors to TUs
sigma_to_rnap_dict = transcription.sigma_factor_complex_to_rna_polymerase_dict
for tu_id in tu_df.index:
    transcription_data = me.process_data.get_by_id(tu_id)
    sigma = tu_df.sigma[tu_id]
    rna_polymerase = sigma_to_rnap_dict[sigma]
    transcription_data.RNA_polymerase = rna_polymerase

#### Degradosome (both for RNA degradation and RNA splicing)

All new data contained in **transcription.py**

In [21]:
me.add_metabolites([cobrame.Complex('RNA_degradosome')])
data = cobrame.ComplexData('RNA_degradosome', me)
for subunit, value in transcription.rna_degradosome.items():
    data.stoichiometry[subunit] = value
data.create_complex_formation(verbose=False)

# Used for RNA splicing
data = cobrame.SubreactionData('RNA_degradation_machine', me)
data.enzyme = 'RNA_degradosome'

data = cobrame.SubreactionData('RNA_degradation_atp_requirement', me)
# .25 water equivaltent for atp hydrolysis per nucleotide
data.stoichiometry = {'atp_c': -.25, 'h2o_c': -.25, 'adp_c': .25,
                      'pi_c': .25, 'h_c': .25}

transcription.add_rna_splicing(me)

------
## Part 3: Add remaining modifications
rRNA modifications handled in *add_ribosome*

### 1) Add complex modifications
*complex_modification_dict* from **protein_modification.text**, above

The rest of the new data contained in **ecolime/modifications.py**

In [22]:
for complex_id, info in complex_modification_dict.items():
    modifications = {}
    for mod, value in info['modifications'].items():
        # stoichiometry of modification determined in 
        # subreaction_data.stoichiometry
        
        modifications['mod_' + mod] = abs(value)
    me.process_data.get_by_id(complex_id).subreactions = modifications

# Adds modification data for more complicated enzyme modifications
# (ie, iron sulfur cluster modification)
ecolime.modifications.add_modification_procedures(me)

In [23]:
with open("./me_models/iBS_checkpoint.pickle", "wb") as outfile:
    pickle.dump(me, outfile)

In [24]:
with open("./me_models/iBS_checkpoint.pickle", "rb") as outfile:
    me = pickle.load(outfile)

In [25]:
# add formation reactions for each of the ComplexDatas
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if 'Oxidized' not in cplx_data.id: 
        if formation:
            formation.update()
        else:
            cplx_data.create_complex_formation()

Created <Metabolite 3fe4s_c at 0x7f2ba280c128> in <ComplexFormation formation_CPLX8J2-2_mod_4fe4s_mod_3fe4s_mod_bmocogdp at 0x7f2b978f0128>


/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:256 UserWarning: met (cu_c) does not have formula
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:256 UserWarning: met (mn2_c) does not have formula
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:256 UserWarning: met (zn2_c) does not have formula
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:256 UserWarning: met (cbl1_c) does not have formula
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:256 UserWarning: met (ca2_c) does not have formula
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:256 UserWarning: met (cobalt2_c) does not have formula
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:256 UserWarning: met (ni2_c) does not have formula
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:256 UserWarning: met (fe2_c) does not have formula
/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:256 UserWarning: met (pheme_c) does not have formula
/home/jt/UCSD/cobrame-

### 2) Add tRNA mods and asocciate them with tRNA charging reactions
New data from:
1. **ecolime/trna_charging.py** (read via *add_tRNA_modification_procedures()*)
2. **post_transcriptional_modification_of_tRNA.txt** (modification types per tRNA)


In [26]:
# Add tRNA modifications to ME-model
ecolime.trna_charging.add_trna_modification_procedures(me)

# trna_modifications = {tRNA_id: {modifications: count}}
trna_modifications=flat_files.get_trna_modification_targets()
for trna in trna_modifications:
    for data in me.process_data.query(trna):
        data.subreactions = trna_modifications[trna]

---
## Part 4: Add remaining subreactions
### 1) Add translation related subreactions
All new data from **ecolime/translation.py**

In [27]:
# add the translation subreaction data objects to model
translation.add_translation_subreactions_to_model(me)

# add translation subreaction data to reactions
methionine_cleaved=translation.methionine_cleaved
folding_dict=translation.folding_dict

for data in me.translation_data:
    locus_id = data.id
    if locus_id in methionine_cleaved:
        data.subreactions['N_terminal_methionine_cleavage'] = 1

    for folding_type in folding_dict:
        if locus_id in folding_dict[folding_type]:
            data.subreactions[folding_type] = 1

    # This block was ran above, but should be ran again to 
    # incorporate any subreactions not added previously
    data.add_initiation_subreactions(
        start_codons=translation.translation_start_codons,
        start_subreactions=set(translation.initiation_subreactions.keys()))
    data.add_elongation_subreactions(
        elongation_subreactions=set(translation.elongation_subreactions.keys()))
    data.add_termination_subreactions(
        translation_terminator_dict=translation.translation_stop_dict)

    # add organism specific subreactions associated with peptide processing
    for subrxn in translation.peptide_processing_subreactions:
        data.subreactions[subrxn] = 1

Adding selenocystein for BSU20040


### 2) Add transcription related subreactions
All new data from **ecolime/transcription.py**

In [28]:
for subreaction in transcription.transcription_subreactions:
    subreaction_data = cobrame.SubreactionData(subreaction, me)
    enzymes = transcription.transcription_subreactions[subreaction]['enzymes']
    subreaction_data.stoichiometry = transcription.transcription_subreactions[subreaction]['stoich']
    subreaction_data.enzyme = enzymes

for transcription_data in me.transcription_data:
    # Assume false if not in tu_df
    rho_dependent = tu_df.rho_dependent.get(transcription_data.id, False)
    rho = 'dependent' if rho_dependent else 'independent'
    stable = 'stable' if transcription_data.codes_stable_rna else 'normal'
    transcription_data.subreactions['Transcription_%s_rho_%s' % (stable,
                                                                 rho)] = 1

----
## Part 5: Add in translocation

New data from:
1. **peptide_compartment_and_pathways.txt** (Protein compartment and translocation pathway for each membrane complex)
2. **ecolime/translocation.py** (definitions of each translocation pathway)

In [29]:
# Add TranslocationData
transloc = pandas.read_csv(join(flat_files.ecoli_files_dir,"peptide_compartment_and_pathways.txt"),
                           sep='\t', comment="#")
for pathway, info in ecolime.translocation.pathway.items():
    if 'alt' not in pathway:
        transloc_data = cobrame.TranslocationData(pathway + '_translocation', me)
    else:
        transloc_data = cobrame.TranslocationData(pathway.replace('_alt', '_translocation_alt'), me)
    transloc_data.enzyme_dict = info['enzymes']
    transloc_data.keff = info['keff']
    transloc_data.length_dependent_energy = info['length_dependent_energy']
    transloc_data.stoichiometry = info['stoichiometry']

# Associate data and add translocation reactions
ecolime.translocation.add_translocation_pathways(me, transloc, membrane_constraints = False)

# Update stoichiometry of membrane complexes
# new_stoich = {complex_id: protein_w_compartment}
new_stoich = defaultdict(dict)
for cplx, row in transloc.set_index('Complex').iterrows():
    if cplx == 'EG10544-MONOMER':
        continue
    protein = row.Protein.split('(')[0] + '_' + row.Protein_compartment
    value = me.process_data.get_by_id(cplx).stoichiometry['protein_' + row.Protein.split('(')[0]]
    new_stoich[cplx]['protein_' + protein] = float(value)

for cplx, stoich in new_stoich.items():
    complex_data = me.process_data.get_by_id(cplx)
    for met, value in stoich.items():
        complex_data.stoichiometry.pop(met[0:16])
        complex_data.stoichiometry[met] = value
        complex_data.formation.update()
    # Complex ids in protein compartment file doesn't include mods
    # Some have multiple alternative modifications so must loop through these
    for complex_data in me.process_data.query(cplx + '_mod_'):
        for met, value in stoich.items():
            complex_data.stoichiometry.pop(met[0:16])
            complex_data.stoichiometry[met] = value
            complex_data.formation.update()

---
## Part 6: Add Cell Wall Components
All new data from **ecolime/translocation.py**

In [30]:
compartment_dict = {}
for prot, compartment in transloc.set_index('Protein').Protein_compartment.to_dict().items():
    compartment_dict[prot.split('(')[0]] = compartment

#### Add lipid modification SubreactionData

In [31]:
# Bsub m-model doest not have pe and pg metabolites

#### Braun's lipoprotein demand
Metabolites and coefficients as defined in [Liu et al 2014](http://bmcsystbiol.biomedcentral.com/articles/10.1186/s12918-014-0110-6)

#### Correct complex formation IDs if they contain lipoproteins

In [32]:
# Bsub m-model doest not have murein

-----
## Part 7: Set keffs

Either entirely based on SASA or using fit keffs from [Ebrahim et al 2016](https://www.ncbi.nlm.nih.gov/pubmed/27782110?dopt=Abstract)

In [33]:
## Get median sasa
from cobrame.core.component import Complex
import numpy as np

sasa_list = []
for met in me.metabolites:
    cplx_sasa = 0.
    if not isinstance(met, Complex):
        continue
    MW = met.formula_weight
    if not MW:
        MW = 0
        print(met.id, ' has no formula')
    cplx_sasa += MW ** (3. / 4)
    sasa_list.append(cplx_sasa)
median_sasa = np.median(np.array(sasa_list))

BSU15920-MONOMER_mod_pan4p  has no formula
BSU28500-MONOMER_mod_Oxidized  has no formula
BSU29679-MONOMER  has no formula
MONOMER8J2-7  has no formula
MONOMER8J2-49  has no formula
MONOMER8J2-60  has no formula
MONOMER8J2-48  has no formula
MONOMER8J2-47  has no formula
MONOMER8J2-46  has no formula
MONOMER8J2-45  has no formula
MONOMER8J2-58  has no formula
MONOMER8J2-59  has no formula
MONOMER8J2-50  has no formula
MONOMER8J2-52  has no formula
MONOMER8J2-55  has no formula
MONOMER8J2-56  has no formula
MONOMER8J2-57  has no formula
BSU23060-MONOMER  has no formula
BSU07540-MONOMER  has no formula


cobra/core/Metabolite.py:97 UserWarning: The element 'X' does not appear in the peridic table


In [34]:
reaction_median_keffs = pandas.read_csv(flat_files.fixpath('reaction_median_keffs.txt'), sep='\t',
                              names=['Reaction', 'Complex', 'Median_Keff'])
keff_list = []
for key, row in reaction_median_keffs.iterrows():
    reaction_id = row['Reaction']
    complex_id = row['Complex']
    median_keff = row['Median_Keff']
    
    real_reaction_id = reaction_id + ''
    
    molecular_weight = me.metabolites.get_by_id(complex_id).formula_weight
    sasa = molecular_weight ** (3./4)
    keff = sasa * median_keff / median_sasa
    
    if keff > 3000:
        keff = 3000.
    elif keff < .01:
        keff = .01
    keff_list.append(keff)
    try:
        me.reactions.get_by_id(reaction_id).keff = keff
        me.reactions.get_by_id(reaction_id).update()
    except:
        continue
    

-----
## Part 8: Model updates and corrections

In [35]:
# Add reaction subsystems from iJO to model
for rxn in ijo.reactions:
    if rxn.id in me.process_data:
        data = me.process_data.get_by_id(rxn.id)
    else:
        continue
    for r in data.parent_reactions:
        r.subsystem = rxn.subsystem

#### Add enzymatic coupling for "carriers"
These are enzyme complexes that act as metabolites in a metabolic reaction (i.e. are metabolites in iJO1366)

In [36]:
for data in me.stoichiometric_data:
    if data.id == 'dummy_reaction':
        continue
        
    for met, value in data.stoichiometry.items():
        if not isinstance(me.metabolites.get_by_id(met), cobrame.Complex) or value > 0:
            continue
            
        subreaction_id = met + '_carrier_activity'
        if subreaction_id not in me.process_data:           
            sub = cobrame.SubreactionData(met + '_carrier_activity', me)
            sub.enzyme = met
        data.subreactions[subreaction_id] = abs(value)

----
## Part 9: Update and save

In [37]:
me.reactions.biomass_dilution.objective_coefficient = 1.
me.reactions.EX_glc__D_e.lower_bound = -1000
me.reactions.EX_o2_e.lower_bound = -1000
me.ngam = 9.
me.gam = 105.053483
me.unmodeled_protein_fraction = .36

me.reactions.BIOMASS_BS_10_FWD_CPLX_dummy.lower_bound = 0
me.reactions.BIOMASS_BS_10_FWD_CPLX_dummy.upper_bound = 0
me.reactions.ATPM_FWD_CPLX_dummy.lower_bound = 0
me.reactions.ATPM_FWD_CPLX_dummy.upper_bound = 0

me.metabolites.get_by_id('mn2_c').formula = 'Mn2'
me.metabolites.get_by_id('mg2_c').formula = 'Mg2'
me.metabolites.get_by_id('ca2_c').formula = 'Ca2'
me.metabolites.get_by_id('cu2_c').formula = 'Cu2'

In [38]:
me.update()

Created <Metabolite Oxidized_c at 0x7f2b969f0cc0> in <ComplexFormation formation_BSU33480-MONOMER_mod_Oxidized at 0x7f2b97956ba8>


/home/jt/UCSD/cobrame-master/cobrame/core/processdata.py:256 UserWarning: met (Oxidized_c) does not have formula


In [39]:
me.prune()

### Add remaining complex formulas and compartments to model

In [40]:
# Update a second time to incorporate all of the metabolite formulas corectly
for r in me.reactions.query('formation_'):
    r.update()

# Update complex formulas
df = pandas.read_table(join(flat_files.ecoli_files_dir, 'modification.txt'),
                            names=['mod', 'formula','na'])
df = df.drop('na', axis=1).set_index('mod').dropna(how='any')
modification_formulas = df.T.to_dict()
formulas.add_remaining_complex_formulas(me, modification_formulas)

# Update reactions affected by formula update 
for r in me.reactions.query('_mod_lipo'):
    r.update()
for r in me.reactions.query('_mod_glycyl'):
    r.update()
    
# add metabolite compartments
compartments.add_compartments_to_model(me)

### Remaining fields

In [41]:
##
current_gene_ids = [gene.id for gene in me.genes]
for met in me.metabolites:
    met_id = met.id
    if 'protein_BSU' in met_id:
        gene_id = met_id.split('_')[1]
        if gene_id and gene_id not in current_gene_ids:
            try:
                print(gene_id)
                gene = cobra.Gene(gene_id)
                me.genes.append(gene)
            except:
                pass

##
me.reactions.BIOMASS_BS_10_FWD_CPLX_dummy.lower_bound = 0
me.reactions.BIOMASS_BS_10_FWD_CPLX_dummy.upper_bound = 0
me.reactions.ATPM_FWD_CPLX_dummy.lower_bound = 0
me.reactions.ATPM_FWD_CPLX_dummy.upper_bound = 0

## Homogenize exchange reactions
for rxn in me.reactions:
    if 'EX_' in rxn.id:
        try:
            lb = ijo.reactions.get_by_id(rxn.id).lower_bound
            ub = ijo.reactions.get_by_id(rxn.id).upper_bound
            rxn.lower_bound = lb
            rxn.upper_bound = ub
        except:
            print(rxn.id, 'not in m_model')

BSU00090
BSU00110
BSU00120
BSU00130
BSU00140
BSU00150
BSU00180
BSU00270
BSU00280
BSU00380
BSU00410
BSU00460
BSU00500
BSU00510
BSU00550
BSU00680
BSU00730
BSU00740
BSU00750
BSU00760
BSU00770
BSU00780
BSU00790
BSU00810
BSU00820
BSU00900
BSU00910
BSU00930
BSU00940
BSU00980
BSU01000
BSU01010
BSU01070
BSU01080
BSU01100
BSU01110
BSU01120
BSU01130
BSU01150
BSU01200
BSU01220
BSU01230
BSU01250
BSU01260
BSU01290
BSU01300
BSU01310
BSU01320
BSU01330
BSU01340
BSU01350
BSU01360
BSU01370
BSU01380
BSU01390
BSU01400
BSU01410
BSU01420
BSU01430
BSU01500
BSU01530
BSU01580
BSU01610
BSU01620
BSU01630
BSU01680
BSU01730
BSU01770
BSU01780
BSU02090
BSU02130
BSU02140
BSU02230
BSU02270
BSU02290
BSU02340
BSU02350
BSU02360
BSU02390
BSU02400
BSU02420
BSU02430
BSU02460
BSU02480
BSU02490
BSU02510
BSU02630
BSU02640
BSU02750
BSU02760
BSU02850
BSU02860
BSU02870
BSU02980
BSU02990
BSU03000
BSU03010
BSU03040
BSU03050
BSU03060
BSU03070
BSU03130
BSU03150
BSU03200
BSU03220
BSU03270
BSU03280
BSU03290
BSU03300
BSU03310
BSU03320
B

In [42]:
with open("./me_models/iYO844b.pickle", "wb") as outfile:
    pickle.dump(me, outfile)

In [43]:
with open("./me_models/iYO844b.pickle", "rb") as outfile:
    me = pickle.load(outfile)

## New fixes

In [44]:
# Feb 12, 2020
# Model was synthesizing extra lipids to compensate and reach maximum growth rate.
# Fix: Set lb = ub = mass fraction of lipids
rxns = get_reactions_of_met(me,'lipid_biomass', s=1, verbose=False)
for rxn in rxns:
    rxn.upper_bound = mu
_=get_reactions_of_met(me,'lipid_biomass', s=1, verbose=True)

( Demand_pgly_BS_c mu mu ) 	 0.000176 pgly_BS_c <=> 0.012548534412479996 lipid_biomass
( Demand_lipo1_24_BS_c mu mu ) 	 7e-06 lipo1_24_BS_c <=> 0.00591416142296 lipid_biomass
( Demand_d12dg_BS_c mu mu ) 	 0.00011 d12dg_BS_c <=> 0.0097263716968 lipid_biomass
( Demand_gtca2_45_BS_c mu mu ) 	 0.002347 gtca2_45_BS_c <=> 0.026715503385342003 lipid_biomass
( Demand_lysylpgly_BS_c mu mu ) 	 2.2e-05 lysylpgly_BS_c <=> 0.0018527632855599998 lipid_biomass
( Demand_lipo3_24_BS_c mu mu ) 	 1.8e-05 lipo3_24_BS_c <=> 0.01205219511504 lipid_biomass
( Demand_gtca1_45_BS_c mu mu ) 	 0.003624 gtca1_45_BS_c <=> 0.026722062249264 lipid_biomass
( Demand_gtca3_45_BS_c mu mu ) 	 0.001819 gtca3_45_BS_c <=> 0.026717435194354 lipid_biomass
( Demand_t12dg_BS_c mu mu ) 	 6.6e-05 t12dg_BS_c <=> 0.006905950978080001 lipid_biomass
( Demand_m12dg_BS_c mu mu ) 	 8.6e-05 m12dg_BS_c <=> 0.00620984507568 lipid_biomass
( Demand_cdlp_BS_c mu mu ) 	 5e-06 cdlp_BS_c <=> 0.0006669379998 lipid_biomass
( Demand_tcam_BS_c mu mu 

In [45]:
solve_me_model(me, max_mu = 0.5, min_mu = .05, using_soplex=False, precision = 1e-6)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 212.524586 seconds
Finished substituting S,lb,ub in 9.029733 seconds
Finished makeME_LP in 1.079063 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.25 0.25 1
Finished substituting S,lb,ub in 9.777054 seconds
Finished makeME_LP in 1.075727 seconds
Getting MINOS parameters from ME_NLP...
2 0.0 0.0 0.125 0.125 1
Finished substituting S,lb,ub in 9.068559 seconds
Finished makeME_LP in 1.088095 seconds
Getting MINOS parameters from ME_NLP...
3 0.0625 0.0625 0.125 0.0625 optimal
Finished substituting S,lb,ub in 8.932621 seconds
Finished makeME_LP in 1.085652 seconds
Getting MINOS parameters from ME_NLP...
4 0.09375 0.09375 0.125 0.09375 optimal
Finished substituting S,lb,ub in 8.774667 seconds
Finished makeME_LP in 1.093390 seconds
Getting MINOS parameters from ME_NLP...
5 0.09375 0.09375 0.109375 0.109375 1
Finished substituting S,lb,ub in 8.775150 seconds
Finished makeME_LP in 1.082010 seconds
Gettin

In [46]:
with open("./me_models/iYO844b.pickle", "wb") as outfile:
    pickle.dump(me, outfile)

In [47]:
save_json_me_model(me, './me_models/iBS.json')

KeyError: 'Oxidized_c'

In [ ]:
with open("./me_models/final/iYO844b.pickle", "wb") as outfile:
    pickle.dump(me, outfile)

In [ ]:
#mets = ['cl', 'sheme', 'mg2', 'ni2', 'thmpp', 'cu', 'nad', 'ca2', 'fmn', 'zn2', 'pydx5p', 'pi', 'pheme',
# 'bmocogdp', '4fe4s', 'mn2', 'cbl1', 'cu2', 'k', 'cobalt2', '2fe2s', 'fe2']
mets = ['glc__D']
met_ids = [met+'_c' for met in mets]

In [ ]:
gap_mets = brute_force_check(me,met_ids,'biomass_dilution',0.02)

In [ ]:
me_ = me.copy()

In [ ]:
from qminospy import me2

In [ ]:
# This adds exchange reactions for metabolites not contained in iJO1366
# Some of these cannot be produced by the model so they are added here
exchange_list = ['4fe4s_c','2fe2s_c','udcpp_c','pydx5p_c','3fe4s','cl_c','sheme_c','cu_c','mn2_c',
                 'bmocogdp_c','dpm_c','thmpp_c','zn2_c','cbl1_c','cobalt2_c']

for met_id in exchange_list:
    met_base = met_id.split('_')[0]
    
    if hasattr(me.metabolites,'{}_c'.format(met_base)):
        met_c = me.metabolites.get_by_id('{}_c'.format(met_base))
    else:
        met_c = cobrame.Metabolite('{}_c'.format(met_base))
        
    if hasattr(me.metabolites,'{}_e'.format(met_base)):
        met_e = me.metabolites.get_by_id('{}_e'.format(met_base))
    else:
        met_e = cobrame.Metabolite('{}_e'.format(met_base))
        
    ex_id = 'EX_{}_e'.format(met_base)
    if hasattr(me.reactions,ex_id):
        ex_r = me.reactions.get_by_id(ex_id)
    else:
        ex_r = cobrame.MEReaction(ex_id)
        ex_r.add_metabolites({met_e:-1})
        me.add_reaction(ex_r)
    t_r = cobrame.MEReaction('{}_t'.format(met_base))
    t_r.add_metabolites({met_e:-1,met_c:1})
    me.add_reaction(t_r)
    ex_r.upper_bound = 0 # Avoid secretion of this metabolites. These reactions are
                         # only meant to supply cofactors for which no biosynthetic 
                         # pathway has been defined in the model.
    ex_r.lower_bound = -1000